In [1]:
pip install opencv-python face_recognition Pillow


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
import sqlite3
import face_recognition
import os
from datetime import datetime
from PIL import Image, ImageTk
import csv

# Database functions
def create_database():
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE NOT NULL,
                        password TEXT NOT NULL
                      )''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS attendance (
                        attendance_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT NOT NULL,
                        timestamp TEXT NOT NULL
                      )''')

    conn.commit()
    conn.close()

def add_user(username, password):
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    try:
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
        conn.commit()
        print(f"User '{username}' added successfully!")
    except sqlite3.IntegrityError:
        print(f"Error: User '{username}' already exists.")

    conn.close()

def validate_login(username, password):
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
    user = cursor.fetchone()
    
    conn.close()
    return user

def mark_attendance(username):
    now = datetime.now()
    date_time = now.strftime("%Y-%m-%d %H:%M:%S")

    # Connect to the SQLite database and store attendance
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('INSERT INTO attendance (username, timestamp) VALUES (?, ?)', (username, date_time))
    conn.commit()
    conn.close()

    # Write attendance to CSV file
    csv_file_path = 'attendance_sheet.csv'
    with open(csv_file_path, mode='a', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow([username, date_time])
    
    print(f"Attendance marked for {username} at {date_time} and recorded in {csv_file_path}.")

# Function to load known faces
def load_known_faces():
    known_faces = {}
    known_face_encodings = []
    
    for filename in os.listdir("known_faces"):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            user_name = os.path.splitext(filename)[0]
            image_path = os.path.join("known_faces", filename)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_faces[user_name] = encoding
            known_face_encodings.append(encoding)
    
    return known_faces, known_face_encodings

# Function to open the attendance window with face detection
def open_attendance_window(username):
    root.withdraw()

    attendance_window = tk.Toplevel()
    attendance_window.title("Mark Attendance")
    attendance_window.geometry("400x300")

    label = tk.Label(attendance_window, text="Please show your face to mark attendance", font=("Arial", 14))
    label.pack(pady=20)

    video_label = tk.Label(attendance_window)
    video_label.pack(pady=10)

    known_faces, known_face_encodings = load_known_faces()

    def start_webcam():
        cap = cv2.VideoCapture(0)

        if not cap.isOpened():
            messagebox.showerror("Error", "Unable to access the camera")
            return

        def update_frame():
            ret, frame = cap.read()
            if ret:
                # Convert the frame to RGB for face_recognition
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Find all face locations and encodings in the current frame
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                # Loop through each face found in the frame
                for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

                    name = "Unknown"
                    # If a match was found, get the name
                    if True in matches:
                        first_match_index = matches.index(True)
                        name = list(known_faces.keys())[first_match_index]

                    # Draw a rectangle around the face
                    cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                    cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

                    # If the recognized face matches the logged-in user, mark attendance
                    if name == username:
                        mark_attendance(username)
                        messagebox.showinfo("Attendance", f"Face verified! Attendance marked for {username}!")
                        cap.release()
                        attendance_window.destroy()

                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(img)
                img = ImageTk.PhotoImage(img)

                video_label.imgtk = img
                video_label.configure(image=img)

                video_label.after(10, update_frame)

        update_frame()

    start_webcam()

# Create the main login window
root = tk.Tk()
root.title("Login Page")
root.geometry("300x200")

# Set the background color to red
root.config(bg="red")

# Create and position the labels and entry widgets for username and password
label_username = tk.Label(root, text="Username", bg="red")
label_username.pack(pady=10)
entry_username = tk.Entry(root)
entry_username.pack(pady=5)

label_password = tk.Label(root, text="Password", bg="red")
label_password.pack(pady=10)
entry_password = tk.Entry(root, show="*")
entry_password.pack(pady=5)

def handle_login():
    username = entry_username.get()
    password = entry_password.get()
    
    user = validate_login(username, password)
    
    if user:
        messagebox.showinfo("Login Info", f"Welcome, {username}!")
        open_attendance_window(username)  # Pass the username for attendance
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")

# Create and position the login button
button_login = tk.Button(root, text="Login", command=handle_login)
button_login.pack(pady=20)

# Initialize the database
create_database()

# Run the Tkinter event loop
root.mainloop()


Attendance marked for karthik at 2024-12-10 00:32:12 and recorded in attendance_sheet.csv.


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\karth\anaconda3\envs\karthikattendance\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\karth\AppData\Local\Temp\ipykernel_6408\2115840897.py", line 184, in handle_login
    open_attendance_window(username)  # Pass the username for attendance
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\karth\AppData\Local\Temp\ipykernel_6408\2115840897.py", line 155, in open_attendance_window
    start_webcam()
  File "C:\Users\karth\AppData\Local\Temp\ipykernel_6408\2115840897.py", line 153, in start_webcam
    update_frame()
  File "C:\Users\karth\AppData\Local\Temp\ipykernel_6408\2115840897.py", line 149, in update_frame
    video_label.configure(image=img)
  File "C:\Users\karth\anaconda3\envs\karthikattendance\Lib\tkinter\__init__.py", line 1721, in configure
    return self._configure('configure', cnf, kw)
           ^^^^^^^^^^^^^^^^^

In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
import sqlite3
import face_recognition
import os
from datetime import datetime
from PIL import Image, ImageTk
import csv


# Ensure 'known_faces' directory exists
if not os.path.exists("known_faces"):
    os.makedirs("known_faces")


# Database functions
def create_database():
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE NOT NULL,
                        password TEXT NOT NULL
                      )''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS attendance (
                        attendance_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT NOT NULL,
                        timestamp TEXT NOT NULL
                      )''')

    conn.commit()
    conn.close()

def add_user(username, password):
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    try:
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
        conn.commit()
        print(f"User '{username}' added successfully!")
    except sqlite3.IntegrityError:
        print(f"Error: User '{username}' already exists.")

    conn.close()

def validate_login(username, password):
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
    user = cursor.fetchone()
    
    conn.close()
    return user

def mark_attendance(username):
    now = datetime.now()
    date_time = now.strftime("%Y-%m-%d %H:%M:%S")

    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('INSERT INTO attendance (username, timestamp) VALUES (?, ?)', (username, date_time))
    conn.commit()
    conn.close()

    csv_file_path = 'attendance_sheet.csv'
    with open(csv_file_path, mode='a', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow([username, date_time])
    
    print(f"Attendance marked for {username} at {date_time} and recorded in {csv_file_path}.")

# Function to load known faces
def load_known_faces():
    known_faces = {}
    known_face_encodings = []
    
    for filename in os.listdir("known_faces"):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            user_name = os.path.splitext(filename)[0]
            image_path = os.path.join("known_faces", filename)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_faces[user_name] = encoding
            known_face_encodings.append(encoding)
    
    return known_faces, known_face_encodings

# Function to open the attendance window with face detection
def open_attendance_window(username):
    root.withdraw()

    attendance_window = tk.Toplevel()
    attendance_window.title("Mark Attendance")
    attendance_window.geometry("400x300")

    label = tk.Label(attendance_window, text="Please show your face to mark attendance", font=("Arial", 14))
    label.pack(pady=20)

    video_label = tk.Label(attendance_window)
    video_label.pack(pady=10)

    known_faces, known_face_encodings = load_known_faces()

    def start_webcam():
        cap = cv2.VideoCapture(0)

        if not cap.isOpened():
            messagebox.showerror("Error", "Unable to access the camera")
            return

        def update_frame():
            ret, frame = cap.read()
            if ret:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

                    name = "Unknown"
                    if True in matches:
                        first_match_index = matches.index(True)
                        name = list(known_faces.keys())[first_match_index]

                    cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                    cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

                    if name == username:
                        mark_attendance(username)
                        messagebox.showinfo("Attendance", f"Face verified! Attendance marked for {username}!")
                        cap.release()
                        attendance_window.destroy()

                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(img)
                img = ImageTk.PhotoImage(img)

                video_label.imgtk = img
                video_label.configure(image=img)

                video_label.after(10, update_frame)

        update_frame()

    start_webcam()

# Function to open user registration window
def open_registration_window():
    registration_window = tk.Toplevel(root)
    registration_window.title("Register New User")
    registration_window.geometry("300x250")

    tk.Label(registration_window, text="Enter New Username").pack(pady=10)
    entry_new_username = tk.Entry(registration_window)
    entry_new_username.pack(pady=5)

    tk.Label(registration_window, text="Enter New Password").pack(pady=10)
    entry_new_password = tk.Entry(registration_window, show="*")
    entry_new_password.pack(pady=5)

    def register_user():
        new_username = entry_new_username.get()
        new_password = entry_new_password.get()
        
        if new_username and new_password:
            add_user(new_username, new_password)
            messagebox.showinfo("Registration", f"User '{new_username}' registered successfully!")
            registration_window.destroy()
        else:
            messagebox.showerror("Error", "Please fill both fields")

    tk.Button(registration_window, text="Register", command=register_user).pack(pady=20)

# Create the main login window
root = tk.Tk()
root.title("Login Page")
root.geometry("300x250")

root.config(bg="red")

label_username = tk.Label(root, text="Username", bg="red")
label_username.pack(pady=10)
entry_username = tk.Entry(root)
entry_username.pack(pady=5)

label_password = tk.Label(root, text="Password", bg="red")
label_password.pack(pady=10)
entry_password = tk.Entry(root, show="*")
entry_password.pack(pady=5)

def handle_login():
    username = entry_username.get()
    password = entry_password.get()
    
    user = validate_login(username, password)
    
    if user:
        messagebox.showinfo("Login Info", f"Welcome, {username}!")
        open_attendance_window(username)
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")

button_login = tk.Button(root, text="Login", command=handle_login)
button_login.pack(pady=10)

button_register = tk.Button(root, text="Register New User", command=open_registration_window)
button_register.pack(pady=10)

create_database()
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
import sqlite3
import face_recognition
import os
from datetime import datetime
from PIL import Image, ImageTk
import csv


# Ensure 'known_faces' directory exists
if not os.path.exists("known_faces"):
    os.makedirs("known_faces")


# Database functions
def create_database():
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE NOT NULL,
                        password TEXT NOT NULL
                      )''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS attendance (
                        attendance_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT NOT NULL,
                        timestamp TEXT NOT NULL
                      )''')

    conn.commit()
    conn.close()

def add_user(username, password):
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    try:
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
        conn.commit()
        print(f"User '{username}' added successfully!")
    except sqlite3.IntegrityError:
        print(f"Error: User '{username}' already exists.")

    conn.close()

def validate_login(username, password):
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
    user = cursor.fetchone()
    
    conn.close()
    return user

def mark_attendance(username):
    now = datetime.now()
    date_time = now.strftime("%Y-%m-%d %H:%M:%S")

    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('INSERT INTO attendance (username, timestamp) VALUES (?, ?)', (username, date_time))
    conn.commit()
    conn.close()

    csv_file_path = 'attendance_sheet.csv'
    with open(csv_file_path, mode='a', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow([username, date_time])
    
    print(f"Attendance marked for {username} at {date_time} and recorded in {csv_file_path}.")

# Function to load known faces
def load_known_faces():
    known_faces = {}
    known_face_encodings = []
    
    for filename in os.listdir("known_faces"):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            user_name = os.path.splitext(filename)[0]
            image_path = os.path.join("known_faces", filename)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_faces[user_name] = encoding
            known_face_encodings.append(encoding)
    
    return known_faces, known_face_encodings

# Function to open the attendance window with face detection
def open_attendance_window(username):
    root.withdraw()

    attendance_window = tk.Toplevel()
    attendance_window.title("Mark Attendance")
    attendance_window.geometry("400x300")

    label = tk.Label(attendance_window, text="Please show your face to mark attendance", font=("Arial", 14))
    label.pack(pady=20)

    video_label = tk.Label(attendance_window)
    video_label.pack(pady=10)

    known_faces, known_face_encodings = load_known_faces()

    def start_webcam():
        cap = cv2.VideoCapture(0)

        if not cap.isOpened():
            messagebox.showerror("Error", "Unable to access the camera")
            return

        def update_frame():
            ret, frame = cap.read()
            if ret:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

                    name = "Unknown"
                    if True in matches:
                        first_match_index = matches.index(True)
                        name = list(known_faces.keys())[first_match_index]

                    cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                    cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

                    if name == username:
                        mark_attendance(username)
                        messagebox.showinfo("Attendance", f"Face verified! Attendance marked for {username}!")
                        cap.release()
                        attendance_window.destroy()

                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(img)
                img = ImageTk.PhotoImage(img)

                video_label.imgtk = img
                video_label.configure(image=img)

                video_label.after(10, update_frame)

        update_frame()

    start_webcam()

# Function to open user registration window
def open_registration_window():
    registration_window = tk.Toplevel(root)
    registration_window.title("Register New User")
    registration_window.geometry("300x250")

    tk.Label(registration_window, text="Enter New Username").pack(pady=10)
    entry_new_username = tk.Entry(registration_window)
    entry_new_username.pack(pady=5)

    tk.Label(registration_window, text="Enter New Password").pack(pady=10)
    entry_new_password = tk.Entry(registration_window, show="*")
    entry_new_password.pack(pady=5)

    def register_user():
        new_username = entry_new_username.get()
        new_password = entry_new_password.get()
        
        if new_username and new_password:
            add_user(new_username, new_password)
            messagebox.showinfo("Registration", f"User '{new_username}' registered successfully!")
            registration_window.destroy()
        else:
            messagebox.showerror("Error", "Please fill both fields")

    tk.Button(registration_window, text="Register", command=register_user).pack(pady=20)

# Create the main login window
root = tk.Tk()
root.title("Login Page")
root.geometry("300x250")

root.config(bg="red")

label_username = tk.Label(root, text="Username", bg="red")
label_username.pack(pady=10)
entry_username = tk.Entry(root)
entry_username.pack(pady=5)

label_password = tk.Label(root, text="Password", bg="red")
label_password.pack(pady=10)
entry_password = tk.Entry(root, show="*")
entry_password.pack(pady=5)

def handle_login():
    username = entry_username.get()
    password = entry_password.get()
    
    user = validate_login(username, password)
    
    if user:
        messagebox.showinfo("Login Info", f"Welcome, {username}!")
        open_attendance_window(username)
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")

button_login = tk.Button(root, text="Login", command=handle_login)
button_login.pack(pady=10)

button_register = tk.Button(root, text="Register New User", command=open_registration_window)
button_register.pack(pady=10)

create_database()
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
import sqlite3
import face_recognition
import os
from datetime import datetime
from PIL import Image, ImageTk
import csv


# Ensure 'known_faces' directory exists
if not os.path.exists("known_faces"):
    os.makedirs("known_faces")


# Database functions
def create_database():
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE NOT NULL,
                        password TEXT NOT NULL
                      )''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS attendance (
                        attendance_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT NOT NULL,
                        timestamp TEXT NOT NULL
                      )''')

    conn.commit()
    conn.close()

def add_user(username, password):
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    try:
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
        conn.commit()
        print(f"User '{username}' added successfully!")
    except sqlite3.IntegrityError:
        print(f"Error: User '{username}' already exists.")

    conn.close()

def validate_login(username, password):
    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
    user = cursor.fetchone()
    
    conn.close()
    return user

def mark_attendance(username):
    now = datetime.now()
    date_time = now.strftime("%Y-%m-%d %H:%M:%S")

    conn = sqlite3.connect('attendance_system.db')
    cursor = conn.cursor()

    cursor.execute('INSERT INTO attendance (username, timestamp) VALUES (?, ?)', (username, date_time))
    conn.commit()
    conn.close()

    csv_file_path = 'attendance_sheet.csv'
    with open(csv_file_path, mode='a', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow([username, date_time])
    
    print(f"Attendance marked for {username} at {date_time} and recorded in {csv_file_path}.")

# Function to load known faces
def load_known_faces():
    known_faces = {}
    known_face_encodings = []
    
    for filename in os.listdir("known_faces"):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            user_name = os.path.splitext(filename)[0]
            image_path = os.path.join("known_faces", filename)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_faces[user_name] = encoding
            known_face_encodings.append(encoding)
    
    return known_faces, known_face_encodings

# Function to open the attendance window with face detection
def open_attendance_window(username):
    root.withdraw()

    attendance_window = tk.Toplevel()
    attendance_window.title("Mark Attendance")
    attendance_window.geometry("400x300")

    label = tk.Label(attendance_window, text="Please show your face to mark attendance", font=("Arial", 14))
    label.pack(pady=20)

    video_label = tk.Label(attendance_window)
    video_label.pack(pady=10)

    known_faces, known_face_encodings = load_known_faces()

    def start_webcam():
        cap = cv2.VideoCapture(0)

        if not cap.isOpened():
            messagebox.showerror("Error", "Unable to access the camera")
            return

        def update_frame():
            ret, frame = cap.read()
            if ret:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

                    name = "Unknown"
                    if True in matches:
                        first_match_index = matches.index(True)
                        name = list(known_faces.keys())[first_match_index]

                    # Draw rectangle around face and label it
                    cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                    cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

                    if name == username:
                        # Mark attendance if the face matches the logged-in user
                        mark_attendance(username)
                        messagebox.showinfo("Attendance", f"Face verified! Attendance marked for {username}!")
                        cap.release()
                        attendance_window.destroy()
                        return
                    elif name != "Unknown":
                        # Alert if the detected face does not match the logged-in user
                        messagebox.showwarning(
                            "Unauthorized Access",
                            f"Detected face '{name}' does not match logged-in user '{username}'!"
                        )

                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(img)
                img = ImageTk.PhotoImage(img)

                video_label.imgtk = img
                video_label.configure(image=img)

                video_label.after(10, update_frame)

        update_frame()

    start_webcam()


# Function to open user registration window
def open_registration_window():
    registration_window = tk.Toplevel(root)
    registration_window.title("Register New User")
    registration_window.geometry("300x250")

    tk.Label(registration_window, text="Enter New Username").pack(pady=10)
    entry_new_username = tk.Entry(registration_window)
    entry_new_username.pack(pady=5)

    tk.Label(registration_window, text="Enter New Password").pack(pady=10)
    entry_new_password = tk.Entry(registration_window, show="*")
    entry_new_password.pack(pady=5)
    def capture_face(username):
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        if ret:
            face_image_path = f"known_faces/{username}.jpg"
            cv2.imwrite(face_image_path, frame)
            messagebox.showinfo("Registration", f"Face captured for {username}!")
        cap.release()

    def register_user():
        new_username = entry_new_username.get()
        new_password = entry_new_password.get()

        if new_username and new_password:
            add_user(new_username, new_password)
            capture_face(new_username)
            registration_window.destroy()
        else:
            messagebox.showerror("Error", "Please fill all fields")

    tk.Button(registration_window, text="Register", command=register_user).pack(pady=20)

# Create the main login window
root = tk.Tk()
root.title("Login Page")
root.geometry("300x250")

root.config(bg="red")

label_username = tk.Label(root, text="Username", bg="red")
label_username.pack(pady=10)
entry_username = tk.Entry(root)
entry_username.pack(pady=5)

label_password = tk.Label(root, text="Password", bg="red")
label_password.pack(pady=10)
entry_password = tk.Entry(root, show="*")
entry_password.pack(pady=5)

def handle_login():
    username = entry_username.get()
    password = entry_password.get()
    
    user = validate_login(username, password)
    
    if user:
        messagebox.showinfo("Login Info", f"Welcome, {username}!")
        open_attendance_window(username)
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")

button_login = tk.Button(root, text="Login", command=handle_login)
button_login.pack(pady=10)

button_register = tk.Button(root, text="Register New User", command=open_registration_window)
button_register.pack(pady=10)

create_database()
root.mainloop()


Attendance marked for karthik at 2025-06-25 00:30:45 and recorded in attendance_sheet.csv.
